In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [ ]:
import os, pickle
from multiprocessing import Pool
from Cfg import Cfg
from load_pretrained_model import load_pretrained_model
from RecordingCorpus import RecordingCorpus
from listen_and_transcribe import listen_and_transcribe
from tqdm.auto import tqdm

language='vietnamese'
phase='build'
release='b30'
max_duration=20
C = Cfg('NIST', 16000, language, phase, release)
model = load_pretrained_model(C, 0)
with Pool(16) as pool:
    recordings = RecordingCorpus(C, pool)

tdir= f'transcriptions/{language}/{phase}/{release}'
os.system(f'mkdir -p {tdir}')
translations = []
for artifact in recordings.artifacts:
    key = artifact.key
    (lng,tfn)=key
    print("key", key)
    save_fn=f'{tdir}/transcription_{lng}_{tfn}.pkl'
    if os.path.exists(save_fn):
        print("finished", key)
        continue
    audio = artifact.source.value
    gold=artifact.gold()
    transcript=listen_and_transcribe(C, model, max_duration, gold, audio)
    translations.append((key, transcript))
    with open(save_fn, 'wb') as f:
        pickle.dump(transcript, f)
    print('saved', save_fn)
    print()